<a href="https://colab.research.google.com/github/hbparache/SAR_ARD_opensource_comparison/blob/main/RMSE_MDD_short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import math
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error 

In [2]:
# Connecting to Google Drive
from google.colab import drive
drive.mount ('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
ls

drive/  sample_data/


In [4]:
cd /content/drive/MyDrive/RMSE_prep/MDD

/content/drive/MyDrive/RMSE_prep/MDD


In [5]:
ls

GAMMA_flat_vh.csv   GRD_flat_vv.csv   GTC_steep_vh.csv   ISCE_steep_vv.csv
GAMMA_flat_vv.csv   GRD_steep_vh.csv  GTC_steep_vv.csv   SNAP_flat_vh.csv
GAMMA_steep_vh.csv  GRD_steep_vv.csv  ISCE_flat_vh.csv   SNAP_flat_vv.csv
GAMMA_steep_vv.csv  GTC_flat_vh.csv   ISCE_flat_vv.csv   SNAP_steep_vh.csv
GRD_flat_vh.csv     GTC_flat_vv.csv   ISCE_steep_vh.csv  SNAP_steep_vv.csv


Read in data

In [42]:
#truth = pd.read_csv("GAMMA_flat_vh.csv") # 'truth', always gamma
#truth = pd.read_csv("GAMMA_flat_vv.csv")
#truth = pd.read_csv("GAMMA_steep_vh.csv")
truth = pd.read_csv("GAMMA_steep_vv.csv")

#Swap the predicted values

#pred = pd.read_csv("GRD_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("GRD_flat_vv.csv") 
#pred = pd.read_csv("GRD_steep_vh.csv")
pred = pd.read_csv("GRD_steep_vv.csv")

#pred = pd.read_csv("GTC_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("GTC_flat_vv.csv") 
#pred = pd.read_csv("GTC_steep_vh.csv")
#pred = pd.read_csv("GTC_steep_vv.csv")

#pred = pd.read_csv("SNAP_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("SNAP_flat_vv.csv") 
#pred = pd.read_csv("SNAP_steep_vh.csv")
#pred = pd.read_csv("SNAP_steep_vv.csv")

#pred = pd.read_csv("ISCE_flat_vh.csv") # 'predicted'
#pred = pd.read_csv("ISCE_flat_vv.csv") 
#pred = pd.read_csv("ISCE_steep_vh.csv")
#pred = pd.read_csv("ISCE_steep_vv.csv")

#get basic stats
print(truth['mean'].describe())
print(pred['mean'].describe())

#set "Date" column to be the date type in pandas, and put in a new column
truth['start_date'] = pd.to_datetime(truth['Date'])
truth['start_date'] = truth['start_date'].apply(lambda t: t.replace(second=0, minute=0, hour=0)) #remove seconds bc these won't match (Gamma & ISCE products won't have)
pred['start_date'] = pd.to_datetime(pred['Date'])
pred['start_date'] = pred['start_date'].apply(lambda t: t.replace(second=0, minute=0, hour=0)) #remove seconds bc these won't match (Gamma & ISCE products won't have)

#check for NaN values that may cause issues later.
print(truth['.geo'].isnull().values.any())
print(truth['start_date'].isnull().values.any())
print(pred['.geo'].isnull().values.any())
print(pred['start_date'].isnull().values.any())

#.geo and start_date columns tog should create a unique identifier for each data point (point location with mean from one day)
truth["new_index"] = truth["start_date"].map(str) + truth[".geo"].map(str)
pred["new_index"] = pred["start_date"].map(str) + pred[".geo"].map(str)

truth = truth.set_index("new_index")
pred = pred.set_index("new_index")

#combined dataframe
combo = truth.join(pred, lsuffix='_truth', rsuffix='_pred')
combo = combo[['mean_truth', 'mean_pred']]
print(combo) #235308 rows x 2 columns
combo.isnull().values.any() #yes null values
combo = combo.dropna()
print(combo.isnull().values.any()) #no null values

# Calculation of RMSE
RMSE = math.sqrt(mean_squared_error(combo['mean_truth'], combo['mean_pred'])) 
print ("Gamma to [product]: RMSE (pw) ", RMSE)

#Convert power to db: dB= 10* log (DN).
x = math.log(RMSE)
y = x*10
print("RMSE (dB)", y)

#pearsonr
print("Pearson R")
r, p = pearsonr(combo['mean_truth'], combo['mean_pred'])
print (r)
print (p)

count    227962.000000
mean          0.244631
std           0.196799
min           0.000055
25%           0.136890
50%           0.207383
75%           0.304466
max          14.562613
Name: mean, dtype: float64
count    228509.000000
mean          0.247898
std           0.413309
min           0.000019
25%           0.109023
50%           0.169237
75%           0.257008
max          28.604059
Name: mean, dtype: float64
False
False
False
False
                                                    mean_truth  mean_pred
new_index                                                                
2019-01-03 00:00:00{"type":"Point","coordinates...    0.206158   0.151205
2019-01-03 00:00:00{"type":"Point","coordinates...    0.216796   0.113468
2019-01-03 00:00:00{"type":"Point","coordinates...    0.252283   0.181730
2019-01-03 00:00:00{"type":"Point","coordinates...    0.895433   0.643891
2019-01-03 00:00:00{"type":"Point","coordinates...    0.301156   0.192945
...                                 